In [80]:
from deepface import DeepFace
import os
import matplotlib.pyplot as plt
from retinaface import RetinaFace
import matplotlib.image as mpimg
import time
import numpy as np
import json
import requests
import pandas as pd

In [4]:
path = "./faces"
images = os.listdir(path)
len(images)

300

In [62]:
backends = [
  'opencv', 
  'ssd',
  'mtcnn', 
  'fastmtcnn',
  'retinaface',
  'yolov8',
  'yunet',
  'centerface',
]
detect_times = {key: 0 for key in backends}
faces_detected = {backend: {} for backend in backends}
for backend in backends:
    for image in images:
        faces_detected[backend][image] = 0 

In [64]:
for backend in backends:
    print("model: ", backend)
    t = 0
    t1 = time.time()
    for image in images:
        try:
            faces = DeepFace.detectFace(
                img_path = f"{path}/{image}",
                detector_backend = backend,
                enforce_detection = True
            )
            faces_detected[backend][image] = 1
        except:
            faces_detected[backend][image] = 0
    t2 = time.time()
    t += t2-t1
    detect_times[backend] = t
    print("time: ", t)

model:  opencv
24-05-20 20:35:23 - ⚠️ Function detectFace is deprecated. Use extract_faces instead.
24-05-20 20:35:25 - ⚠️ Function detectFace is deprecated. Use extract_faces instead.
24-05-20 20:35:26 - ⚠️ Function detectFace is deprecated. Use extract_faces instead.
24-05-20 20:35:27 - ⚠️ Function detectFace is deprecated. Use extract_faces instead.
24-05-20 20:35:28 - ⚠️ Function detectFace is deprecated. Use extract_faces instead.
24-05-20 20:35:29 - ⚠️ Function detectFace is deprecated. Use extract_faces instead.
24-05-20 20:35:30 - ⚠️ Function detectFace is deprecated. Use extract_faces instead.
24-05-20 20:35:31 - ⚠️ Function detectFace is deprecated. Use extract_faces instead.
24-05-20 20:35:31 - ⚠️ Function detectFace is deprecated. Use extract_faces instead.
24-05-20 20:35:33 - ⚠️ Function detectFace is deprecated. Use extract_faces instead.
24-05-20 20:35:33 - ⚠️ Function detectFace is deprecated. Use extract_faces instead.
24-05-20 20:35:33 - ⚠️ Function detectFace is depr

In [71]:
# face++
import requests
http_url = 'https://api-us.faceplusplus.com/facepp/v3/detect'
key = '7SNt9yPlw_ITkwUisnXUU43-LTvHwUkD'
secret = 'IdrN36XnjHe88AdFHio6eyWSn7THc-Gb'


def facepp_detection(filepath):
    boundary = '----------%s' % hex(int(time.time() * 1000))
    data = []
    data.append('--%s' % boundary)
    data.append('Content-Disposition: form-data; name="%s"\r\n' % 'api_key')
    data.append(key)
    data.append('--%s' % boundary)
    data.append('Content-Disposition: form-data; name="%s"\r\n' % 'api_secret')
    data.append(secret)
    data.append('--%s' % boundary)
    fr = open(filepath, 'rb')
    data.append('Content-Disposition: form-data; name="%s"; filename=" "' % 'image_file')
    data.append('Content-Type: %s\r\n' % 'application/octet-stream')
    data.append(fr.read())
    fr.close()
    data.append('--%s' % boundary)
    data.append('Content-Disposition: form-data; name="%s"\r\n' % 'return_landmark')
    data.append('1')
    data.append('--%s' % boundary)
    data.append('Content-Disposition: form-data; name="%s"\r\n' % 'return_attributes')
    data.append(
        "gender,age,smiling,headpose,facequality,blur,eyestatus,emotion,ethnicity,beauty,mouthstatus,eyegaze,skinstatus")
    data.append('--%s--\r\n' % boundary)
    
    for i, d in enumerate(data):
        if isinstance(d, str):
            data[i] = d.encode('utf-8')
    
    http_body = b'\r\n'.join(data)
    
    headers = {
        'Content-Type': 'multipart/form-data; boundary=%s' % boundary
    }
    
    try:
        response = requests.post(http_url, data=http_body, headers=headers, verify=True, timeout=5)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx and 5xx)

        qrcont = response.content
        faces = json.loads(qrcont.decode('utf-8'))['faces']
        if len(faces) == 0:
            return 0
        else:
            return 1
    except requests.exceptions.RequestException as e:
        # Print or log the error for debugging
        print(f"Request failed: {e}")
        return 0

In [72]:
faces_detected['facepp'] = {}
for image in images:
    faces_detected['facepp'][image] = 0 

t1 = time.time()
for image in images:
    img_path = f"{path}/{image}"
    if facepp_detection(img_path) == 1:
        faces_detected['facepp'][image] = 1
    else:
        faces_detected['facepp'][image] = 0
    time.sleep(0.34)
t2 = time.time()
t = t2-t1
detect_times['facepp'] = t

Request failed: 400 Client Error: Bad Request for url: https://api-us.faceplusplus.com/facepp/v3/detect
Request failed: ('Connection aborted.', TimeoutError('The write operation timed out'))
Request failed: 400 Client Error: Bad Request for url: https://api-us.faceplusplus.com/facepp/v3/detect
Request failed: ('Connection aborted.', TimeoutError('The write operation timed out'))
Request failed: 400 Client Error: Bad Request for url: https://api-us.faceplusplus.com/facepp/v3/detect


In [73]:
# time for each backend model
detect_times

{'opencv': 334.9292221069336,
 'ssd': 47.220543384552,
 'mtcnn': 582.0917568206787,
 'fastmtcnn': 132.33928537368774,
 'retinaface': 5482.4845905303955,
 'yolov8': 73.21981000900269,
 'yunet': 21.155822277069092,
 'centerface': 279.6671419143677,
 'facepp': 797.8929996490479}

In [89]:
backends = [
  'opencv', 
  'ssd',
  'mtcnn', 
  'fastmtcnn',
  'retinaface',
  'yolov8',
  'yunet',
  'centerface',
  'facepp',
]
confusion_matrices = {backend: {} for backend in backends}
for backend in backends:
    confusion_matrices[backend]['tp'] = 0
    confusion_matrices[backend]['fn'] = 0 
    confusion_matrices[backend]['tn'] = 0 
    confusion_matrices[backend]['fp'] = 0 

In [91]:
for backend in backends:
    for image in images:
        if image.split(".")[0].startswith("noise"):
            if image.split(".")[0].split("_")[1] == 'f':
                if faces_detected[backend][image] == 1:
                    confusion_matrices[backend]['tp'] += 1
                else:
                    confusion_matrices[backend]['fn'] += 1
            else:
                if faces_detected[backend][image] == 1:
                    confusion_matrices[backend]['fp'] += 1
                else:
                    confusion_matrices[backend]['tn'] += 1
        else:
            if faces_detected[backend][image] == 1:
                confusion_matrices[backend]['tp'] += 1
            else:
                confusion_matrices[backend]['fn'] += 1
pd.DataFrame.from_dict(confusion_matrices, orient='index')   

,tp,fn,tn,fp
opencv,187,43,59,11
ssd,183,47,70,0
mtcnn,227,3,53,17
fastmtcnn,227,3,46,24
retinaface,226,4,47,23
yolov8,230,0,45,25
yunet,204,26,70,0
centerface,202,28,64,6
facepp,219,11,58,12


In [92]:
performance = {backend: {} for backend in backends}
for backend in backends:
    performance[backend]['time'] = detect_times[backend]
    performance[backend]['accuracy'] = (confusion_matrices[backend]['tp'] + confusion_matrices[backend]['tn']) / 300
    performance[backend]['precision'] = confusion_matrices[backend]['tp'] / (confusion_matrices[backend]['tp'] + confusion_matrices[backend]['fp'])
    performance[backend]['recall'] = confusion_matrices[backend]['tp'] / (confusion_matrices[backend]['tp'] + confusion_matrices[backend]['fn'])
pd.DataFrame.from_dict(performance, orient='index')

,time,accuracy,precision,recall
opencv,334.929222,0.820000,0.944444,0.813043
ssd,47.220543,0.843333,1.000000,0.795652
mtcnn,582.091757,0.933333,0.930328,0.986957
fastmtcnn,132.339285,0.910000,0.904382,0.986957
retinaface,5482.484591,0.910000,0.907631,0.982609
yolov8,73.219810,0.916667,0.901961,1.000000
yunet,21.155822,0.913333,1.000000,0.886957
centerface,279.667142,0.886667,0.971154,0.878261
facepp,797.893000,0.923333,0.948052,0.952174
